# Introduction to Image Filtering

Adopted from [SciPy 2018 - Image Analysis in Python with SciPy and scikit-image](https://youtu.be/arXiv-TM7DY).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

#Custom Display Parameters in Matplotlib
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
#Create a function for displaying graphs

def displayg(image):
    fig, ax = plt.subplots()
    ax.plot(image)
    ax.margins(y=0.1)

In [ ]:
#Create an array of 100 values (50 zeros, 50 ones) for a signal 
stepsig = np.zeros(100)
stepsig[50:] = 1

In [ ]:
stepsig

In [ ]:
#Confirm array dimension

stepsig.shape

In [ ]:
#View the array values for the signal

displayg(stepsig)

In [ ]:
#Add noise to the signal
np.random.seed(0)
noisysig = (stepsig 
                + np.random.normal(0, 0.35, stepsig.shape))
displayg(noisysig)

In [ ]:
noisysig

In [ ]:
#Create a moving average of the preceeding and following values for the signal
nnmean = (noisysig[:-1]+noisysig[1:]/2)

In [ ]:
displayg(nnmean)

In [ ]:
#Create a 3-value moving average of the preceeding following and current values for the signal
smoothsig3 = ((noisysig[:-2] + noisysig[1:-1] + noisysig[2:]) / 3)
fig, ax = plt.subplots()
ax.plot(noisysig, label = 'Mean of 2')
ax.plot(smoothsig3, label = 'Mean of 3')
ax.legend(loc = 'upper left')

In [ ]:
#Use first pixel as an example
(noisysig[0]+noisysig[1]+noisysig[2])/3

In [ ]:
#Same example pixel using convolution
((noisysig[0]*0.333333333)+(noisysig[1]*0.333333333) +noisysig[2]*0.333333333))

In [ ]:
smoothsig3

In [ ]:
# Create same 3-value moving average using Numpy Convolve function
mean_kernal3 = np.full(3, 1/3)
smooth_signal3p = np.convolve(noisysig, mean_kernal3,
                                 mode = 'valid')
displayg(smooth_signal3p)

In [ ]:
# Using a 11-value moving window

mean_kernal11 = np.full(11, 1/11)
smooth_signal11p = np.convolve(noisysig, mean_kernal11,
                                 mode = 'valid')
fig, ax = plt.subplots()
ax.plot(smooth_signal11p)

In [ ]:
#Mode = Same (uses zeros at the edge to get all 11 samples)
mean_kernal11 = np.full(11, 1/11)
smooth_signal11p = np.convolve(noisysig, mean_kernal11,
                                 mode = 'same')
fig, ax = plt.subplots()
ax.plot(smooth_signal11p)

In [ ]:
import scipy

In [ ]:
#Using Scipy NDimage Convolve
from scipy import ndimage
%matplotlib inline
sciimage = scipy.ndimage.convolve(noisysig, mean_kernal11, mode='reflect')

In [ ]:
fig, ax = plt.subplots()
ax.plot(sciimage)

# Difference Filters

In [ ]:
displayg(stepsig)

In [ ]:
stepsig[50], stepsig[51], stepsig[52]

In [ ]:
((stepsig[50]*-1)+(stepsig[51]*0)+(stepsig[52]*1))

In [ ]:
# Create the Difference Filter using NP.Convolve
result = np.convolve(stepsig, np.array([1,0,-1]), mode = 'valid')

In [ ]:
result.shape

In [ ]:
stepsig

In [ ]:
fig, ax = plt.subplots()
ax.plot(stepsig, label = 'signal')
ax.plot(result, linestyle = 'dashed', label = 'result')
ax.legend(loc = 'upper left')
ax.margins(y=0.1)

**Exercise:** The Gaussian filter with variance $\sigma^2$ is given by:

$$
k_i = \frac{1}{\sqrt{2\pi}\sigma}\exp{\left(-\frac{(x_i - x_0)^2}{2\sigma^2}\right)}
$$

1. Create this filter (for example, with width 9, center 4, sigma 1). (Plot it)
2. Convolve it with the difference filter (with appropriate mode). (Plot the result)
3. Convolve it with the noisy signal. (Plot the result)

In [ ]:
k = 1 / np.sqrt(2*np.pi)*np.exp(-(np.arange(9)- 4)**2/2)

In [ ]:
k

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1,3, figsize = (20,5))
ax[0].plot(k, label = 'Gaussian')

smooth_diff = np.convolve(k, [1, 0, -1], mode = 'full')
ax[1].plot(smooth_diff)

smooth_diff_signal = np.convolve(noisysig, smooth_diff)
ax[2].plot(smooth_diff_signal)

# 2D (Image) Filtering

In [ ]:
import numpy as np

bright_square = np.zeros((7, 7), dtype=float)
bright_square[2:5, 2:5] = 1

In [ ]:
bright_square

In [ ]:
fig, ax = plt.subplots()
ax.imshow(bright_square);

The mean filter
For our first example of a filter, consider the following filtering array, which we'll call a "mean kernel". For each pixel, a kernel defines which neighboring pixels to consider when filtering, and how much to weight those pixels.

### The mean filter

For our first example of a filter, consider the following filtering array, which we'll call a "mean kernel". For each pixel, a kernel defines which neighboring pixels to consider when filtering, and how much to weight those pixels.

In [ ]:
mean_kernel = np.full((3, 3), 1/9)

print(mean_kernel)

Now, let's take our mean kernel and apply it to every pixel of the image.

Applying a (linear) filter essentially means:

Center a kernel on a pixel
Multiply the pixels under that kernel by the values in the kernel
Sum all the those results
Replace the center pixel with the summed result
This process is known as convolution.

In [ ]:
import scipy.ndimage as ndi

%precision 2
print(bright_square)

#Convolve the mean kernal over the bright square image
print(ndi.correlate(bright_square, mean_kernel))

### Practical Example

In [ ]:
from skimage import data

#load image from skimage library
image = data.camera()

#Downsample the image (every 10th pixel)
pixelated = image[::10, ::10]

#Plot the two images:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))
ax0.imshow(image)
ax1.imshow(pixelated) ;

In [ ]:
#create a custom function for displaying the before and after images

from skimage import img_as_float

def imshow_all(*images, titles=None):
    images = [img_as_float(img) for img in images]

    if titles is None:
        titles = [''] * len(images)
    vmin = min(map(np.min, images))
    vmax = max(map(np.max, images))
    ncols = len(images)
    height = 5
    width = height * len(images)
    fig, axes = plt.subplots(nrows=1, ncols=ncols,
                             figsize=(width, height))
    for ax, img, label in zip(axes.ravel(), images, titles):
        ax.imshow(img, vmin=vmin, vmax=vmax)
        ax.set_title(label)

In [ ]:
#apply mean filter to the original downsampled image
filtered = ndi.correlate(pixelated, mean_kernel)

#display the images
imshow_all(pixelated, filtered, titles=['pixelated', 'mean filtered'])

### Gaussian Filter

Gaussian filter
The classic image filter is the Gaussian filter. This is similar to the mean filter, in that it tends to smooth images. The Gaussian filter, however, doesn't weight all values in the neighborhood equally. Instead, pixels closer to the center are weighted more than those farther away.

Incidentally, for reference, let's have a look at what the Gaussian filter actually looks like. Technically, the value of the kernel at a pixel that is $r$ rows and $c$ cols from the center is:

$$
k_{r, c} = \frac{1}{2\pi \sigma^2} \exp{\left(-\frac{r^2 + c^2}{2\sigma^2}\right)}
$$


this value is pretty close to zero for values more than $4\sigma$ away from the center, so practical Gaussian filters are truncated at about $4\sigma$:

In [ ]:
# Rename module so we don't shadow the builtin function
from skimage import filters

smooth_mean = ndi.correlate(bright_square, mean_kernel)
sigma = 0.5
smooth = filters.gaussian(bright_square, sigma)
imshow_all(bright_square, smooth_mean, smooth,
           titles=['original', 'result of mean filter', 'result of gaussian filter'])

In [ ]:
sidelen = 45
#sigma = (sidelen - 1) // 2 // 4
spot = np.zeros((sidelen, sidelen), dtype=float)
spot[sidelen // 2, sidelen // 2] = 1
kernel = filters.gaussian(spot, sigma=4)

imshow_all(spot, kernel / np.max(kernel))

In [ ]:
#apply mean filter to the original downsampled image
filtered = ndi.correlate(pixelated, mean_kernel)
gfiltered = filters.gaussian(pixelated, sigma =1)
#display the images
imshow_all(pixelated, filtered, gfiltered, titles=['pixelated', 'mean filtered', 'Gaussian'])